In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv as csv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
#유한차분법을 이용한 numerical_gradient 함수 

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [3]:
#코드 구현에 필요한 수식

def sigmoid(x):
    return 1 / (1 + np.exp(-x))    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

    
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [4]:
# coding: utf-8

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    # 예측
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        # Affine -> Sigmoid -> Affine -> Softmax
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # 손실함수    
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    # 정확도 측정
    def accuracy(self, x, t):
        y = self.predict(x)
        # axis = 1일때 즉, 행에서 가장 값이 큰 인덱스를 리턴한다.
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [ ]:


# 데이터 읽기
# normalize를 True로 하는 의미 : 픽셀 값을 255로 나눠 0~1 사이 값으로 나타내겠다는 뜻
# one_hot_label을 True로 하는 의미 : 0-9 정수를 원핫벡터로 변환
# flatten은 디폴트가 True 이므로 여기선 생략

from mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수를 10000번으로 설정
train_size = x_train.shape[0] # x_train.shape() = (60000, 784) 이므로 train_size는 60000 이다.
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1 에폭 당 반복 수 (epoch이란, 훈련 데이터를 다 소진하는 하나의 구간을 의미함)
# ex. MNIST에서는 1에폭당 6만장의 훈련 데이터가 사용된다.
# 아래 코드의 의미는 한 에폭당 배치 사이즈만큼 즉 (train_size / batch_size = 600번) 반복된다는 뜻이다.
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득 
    # train_size 이하의 정수 중 batch_size 만큼을 랜덤 추출하라는 뜻
    batch_mask = np.random.choice(train_size, batch_size)
    # 인덱스가 랜덤 추출한 정수에 해당하는 데이터를 추출 
    # -> 학습 데이터를 batch_size 만큼 랜덤하게 추출할 수 있음
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    # 손실함수의 수치 gradient함수에 가중치와 편향을 넣은 결과를 딕셔너리로 리턴받음
    # grad = network.gradient(x_batch, t_batch) -> 역전파 이용
    grad = network.numerical_gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1 에폭 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train acc, test acc | 0.0993, 0.1032


In [ ]:
'''
참고자료
https://frozenca.wordpress.com/2020/11/28/5-neural-networks/
https://m.blog.naver.com/vi_football/221878546500
http://norman3.github.io/prml/docs/chapter05/3.html
https://roadcom.tistory.com/87
http://incredible.ai/artificial-intelligence/2017/01/07/Deep-Learning/
https://yonsodev.tistory.com/11
https://github.com/WegraLee/deep-learning-from-scratch/blob/82457ee673646f58708068842e825530b7fd1640/common/functions.py#L1
https://www.youtube.com/watch?v=seoXj2abFDQ
'''